Datensatz öffnen

In [1]:
import pandas as pd
df = pd.read_csv('movies.csv')
print(df.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Jahr als separates Feature

In [2]:
# Jahr extrahieren in neue Spalte 'year'
df['year'] = df['title'].str.extract(r'\((\d{4})\)')

# Klammerjahr aus dem Titel entfernen
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()

# Ergebnis anzeigen
print(df.head(10))


   movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   
5        6                         Heat   
6        7                      Sabrina   
7        8                 Tom and Huck   
8        9                 Sudden Death   
9       10                    GoldenEye   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  
5                        Action|Crime|Thriller  1995  
6                               Comedy|Romance  1995  
7                           Adventure|Children  1995  
8                              

Prototyp

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Daten vorbereiten
df = pd.read_csv('movies.csv')
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')

# Genre-Vektoren (einmalig!)
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])

# Mapping von Titel zu Index
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# User Input (3 Titel)
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

# Finde die Vektoren der eingegebenen Filme
valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    # Durchschnittlicher Genre-Vektor (mit Umwandlung zu dichten Array)
    user_vector = genre_matrix[valid_indices].mean(axis=0).A.flatten()

    # Ähnlichkeit zwischen User-Vektor und allen Filmen berechnen
    similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix).flatten()

    # Top-N Empfehlungen (ausgenommen Eingaben)
    top_n = 5
    recommendations = similarities.argsort()[::-1]
    recommendations = [i for i in recommendations if i not in valid_indices][:top_n]

    print("\n🎬 Empfehlungen:")
    print(df.iloc[recommendations][['title', 'genres', 'year']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:
⚠️  Film nicht gefunden: jökfdjlsökdajfdskjf

🎬 Empfehlungen:
                                       title  \
61889                              UglyDolls   
30639    Scooby-Doo! Mask of the Blue Falcon   
23228                      The Magic Crystal   
72526              Legends of Valhalla: Thor   
49160  Puss in Book: Trapped in an Epic Tale   

                                            genres  year  
61889  Adventure|Animation|Children|Comedy|Fantasy  2019  
30639  Adventure|Animation|Children|Comedy|Fantasy  2012  
23228  Adventure|Animation|Children|Comedy|Fantasy  2011  
72526  Adventure|Animation|Children|Comedy|Fantasy  2011  
49160  Adventure|Animation|Children|Comedy|Fantasy  2017  


Timemachine

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Daten vorbereiten
df = pd.read_csv('movies.csv')
df['year'] = df['title'].str.extract(r'\((\d{4})\)')
df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True).str.strip()
df['year'] = df['year'].astype('Int64')

# Zeilen ohne Jahr ausschließen
df = df.dropna(subset=['year'])

# Genre-Vektoren (einmalig!)
vectorizer = CountVectorizer(tokenizer=lambda x: x.split('|'))
genre_matrix = vectorizer.fit_transform(df['genres'])

# Mapping von Titel zu Index
title_to_index = {t.lower(): i for i, t in enumerate(df['title'])}

# User Input (3 Titel)
print("Bitte gib 3 Filmtitel ein:")
user_inputs = [input(f"Film {i+1}: ").strip().lower() for i in range(3)]

# Finde die Vektoren der eingegebenen Filme
valid_indices = []
for title in user_inputs:
    if title in title_to_index:
        valid_indices.append(title_to_index[title])
    else:
        print(f"⚠️  Film nicht gefunden: {title}")

if not valid_indices:
    print("❌ Keine gültigen Filme eingegeben. Abbruch.")
else:
    # Durchschnittlicher Genre-Vektor (mit Umwandlung zu dichten Array)
    user_vector = genre_matrix[valid_indices].mean(axis=0).A.flatten()

    # Ähnlichkeit zwischen User-Vektor und allen Filmen berechnen
    similarities = cosine_similarity(user_vector.reshape(1, -1), genre_matrix).flatten()

    # Top-N Empfehlungen (ausgenommen Eingaben)
    top_n = 5
    recommendations = similarities.argsort()[::-1]
    recommendations = [i for i in recommendations if i not in valid_indices][:top_n]

    print("\n🎬 Allgemeine Empfehlungen:")
    print(df.iloc[recommendations][['title', 'genres', 'year']])

    # Time Machine: alte und neue Filme
    old_mask = df['year'] < 2010
    new_mask = df['year'] >= 2010

    df_old = df[old_mask].reset_index(drop=True)
    df_new = df[new_mask].reset_index(drop=True)

    genre_old = genre_matrix[old_mask.values]
    genre_new = genre_matrix[new_mask.values]

    # Ähnlichkeiten berechnen
    sim_old = cosine_similarity(user_vector.reshape(1, -1), genre_old).flatten()
    sim_new = cosine_similarity(user_vector.reshape(1, -1), genre_new).flatten()

    rec_old = sim_old.argsort()[::-1][:top_n]
    rec_new = sim_new.argsort()[::-1][:top_n]

    print("\n🕰️ Empfehlungen aus der Vergangenheit (vor 2010):")
    print(df_old.iloc[rec_old][['title', 'genres', 'year']])

    print("\n🚀 Empfehlungen aus der Zukunft (2010 oder später):")
    print(df_new.iloc[rec_new][['title', 'genres', 'year']])


c:\Users\deicc\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Bitte gib 3 Filmtitel ein:
⚠️  Film nicht gefunden: title
⚠️  Film nicht gefunden: penis

🎬 Allgemeine Empfehlungen:
                      title       genres  year
40517        Cereal Killers  Documentary  2013
17587  End of the Line, The  Documentary  2009
17519         Metrobranding  Documentary  2010
66622              Elephant  Documentary  2020
66619  In My Father’s House  Documentary  2015

🕰️ Empfehlungen aus der Vergangenheit (vor 2010):
                                            title       genres  year
11877  Chronicle of a Summer (Chronique d'un été)  Documentary  1961
48065                     Sapporo Winter Olympics  Documentary  1972
48063       Sydney 2000: Stories of Olympic Glory  Documentary  2001
48062    Salt Lake 2002: Stories of Olympic Glory  Documentary  2003
48061           Lillehammer ’94: 16 Days of Glory  Documentary  1994

🚀 Empfehlungen aus der Zukunft (2010 oder später):
                                                  title       genres  year
26135    